In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pydot
import seaborn as sns
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix,roc_curve,auc , classification_report
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

from PIL import Image
from PIL import Image as pil_image
from time import time
from PIL import ImageDraw
from glob import glob
from tqdm import tqdm
from scipy import misc,ndimage
from scipy.ndimage.interpolation import zoom
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model , model_to_dot
from keras import backend as K
from keras import layers
from keras.preprocessing.image import save_img
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D,SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler
from IPython.display import SVG

In [0]:
#data generation
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [0]:
training_set = train_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/Intel_Image/seg_train',target_size=(150,150),batch_size=8,class_mode = 'categorical')
valid_set = valid_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/Intel_Image/seg_test',target_size=(150,150),batch_size=8,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/Intel_Image/seg_pred',target_size=(150,150),batch_size=8,class_mode = None , )

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 2 images belonging to 1 classes.


In [0]:
batch_size=8
training_set.target_size,valid_set.target_size,test_set.target_size

((150, 150), (150, 150), (150, 150))

In [0]:
train_size = training_set.n
valid_size = valid_set.n
test_size =test_set.n

In [0]:
test_size

0

In [0]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

W0806 00:11:17.108395 140588443891584 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
#custom metrics 
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [0]:
# Compile the model
model.compile(optimizer=SGD(lr=1e-4,momentum=0.99), loss='categorical_crossentropy', metrics=['accuracy',categorical_accuracy, top_2_accuracy, top_3_accuracy])

W0806 00:11:30.205492 140588443891584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [0]:
#train the model
history = model.fit_generator(training_set,steps_per_epoch = train_size/batch_size,epochs = 1 # For the testing,Only using 1 epoch (otherwise will take weeks to train)
                              ,validation_data = valid_set,validation_steps  = valid_size/batch_size)
#evaluating the model
eval = model.evaluate_generator(generator=valid_set,steps=valid_size/batch_size)
print('accuracy=',eval[1])

RuntimeError: ignored

In [0]:
y_pred=model.predict_generator(generator=test_set,steps=valid_size//batch_size)
y_pred=np.argmax(y_pred,axis=1)

In [0]:
print('confusion matrix')
print(confusion_matrix(valid_set.classes,y_pred))
#confusion_matrix
cm=confusion_matrix(valid_set.classes,y_pred)
cm_df=pd.DataFrame(cm,index=['buildings','foresst','glacier','mountain','sea','street'],columns=['buildings','foresst','glacier','mountain','sea','street'])

confusion matrix


ValueError: ignored

In [0]:
plt.figure(figsize=(5,5))
sns.heatmap(cm_df,annot=True)
plt.title('confusion matrix')
plt.ylabel('true label')
plt.xlabel('predicted label')
plt.savefig('confusion')
plt.show()

NameError: ignored

<Figure size 360x360 with 0 Axes>

In [0]:
#classification_report
print('classification report')
target_names=['buildings','foresst','glacier','mountain','sea','street']
print(classification_report(valid_set.classes,y_pred,target_names=target_names))

classification report


ValueError: ignored

In [0]:
#test set prediction
test_set.reset()
pred=model.predict_generator(test_set,steps=test_size/batch_size,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]


In [0]:
#saving the predictions to csv
filenames=test_set.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("vgg19results.csv",index=False)

In [0]:
#plotting accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('modelacc')
plt.show()

In [0]:
#plotting loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('modelloss')
plt.show()
